In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
print 'Library versions used.'
print 'Numpy :{} \nTensorflow :{}'.format(np.version.version, tf.__version__)

Library versions used.
Numpy :1.13.1 
Tensorflow :1.2.1


In [3]:
from tensorflow.examples.tutorials.mnist import input_data
data = input_data.read_data_sets('data/MNIST', one_hot=True)

Extracting data/MNIST/train-images-idx3-ubyte.gz
Extracting data/MNIST/train-labels-idx1-ubyte.gz
Extracting data/MNIST/t10k-images-idx3-ubyte.gz
Extracting data/MNIST/t10k-labels-idx1-ubyte.gz


In [4]:
print 'Data Size'
print 'Training Data \t:', len(data.train.labels)
print 'validation Data :', len(data.validation.labels)
print 'Testing Data \t:', len(data.test.labels)

Data Size
Training Data 	: 55000
validation Data : 5000
Testing Data 	: 10000


In [5]:
image_size = 28
image_size_flatten = image_size * image_size
num_labels = 10
hidden_dim =128

In [6]:
x = tf.placeholder(tf.float32, [None, image_size_flatten], name='x')
y_true = tf.placeholder(tf.int32, [None, num_labels], name='labels')

In [7]:
class input_layer(object):
    def __init__(self, inpt, name):
        with tf.name_scope(name):
            
            self.W = tf.Variable(tf.truncated_normal([image_size, hidden_dim], stddev=0.1), name='input_W')
            self.b = tf.Variable(tf.truncated_normal([hidden_dim], stddev=0.1), name='input_b')

            self.X = tf.reshape(inpt, [-1, image_size])

            self.output = tf.matmul(self.X, self.W) + self.b  #[64*28, 128]

#             self.output = tf.reshape(_output,[-1, hidden_dim])

            tf.summary.histogram("weights", self.W)
            tf.summary.histogram("bias", self.b)

In [8]:
class vanilla_RNN(object):
    def __init__(self,inpt, name):
        with tf.name_scope(name):
            self.W = tf.Variable(tf.truncated_normal([hidden_dim, hidden_dim], stddev=0.1), name='RNN_W')
#             self.b = tf.Variable(tf.random_normal([hidden_dim]), name='RNN_b')                

            def forward_propagation(prev_state, inpt_t):
                current_state = tf.nn.sigmoid(tf.matmul(prev_state, self.W) + inpt_t)
                return current_state

            _output1 = tf.scan(forward_propagation, inpt, initializer=tf.zeros([1, hidden_dim])) #[64*28, 128]
            _output2 = tf.reshape(_output1, [-1, image_size, hidden_dim]) #[64, 28, 128]
            _output3 = tf.transpose(_output2, [1, 0, 2]) #[28, 64, 128]
            _output = tf.unstack(_output3) # [28, 64, 128]
            self.output = _output[-1] #28 x [64, 128]

            tf.summary.histogram("weights", self.W)
#             tf.summary.histogram("bias", self.b)

In [9]:
class LSTMcell(object):
    def __init__(self, inpt, name):
        with tf.name_scope(name):
            
            self.inpt = tf.reshape(inpt,[-1, image_size, hidden_dim])
#             self.inpt = tf.unstack(_inpt, image_size, 1)
            self.lstm_cell = tf.contrib.rnn.BasicLSTMCell(hidden_dim)
            all_state, states = tf.nn.dynamic_rnn(self.lstm_cell, self.inpt, dtype=tf.float32)        
            self.output = states[-1]
            

In [10]:
class output_layer(object):
    def __init__(self, inpt, name):
        with tf.name_scope(name):
            self.W = tf.Variable(tf.truncated_normal([hidden_dim, num_labels], stddev=0.1), name='output_W')
            self.b = tf.Variable(tf.truncated_normal([num_labels], stddev=0.1), name='output_b')
            
            self.output = tf.matmul(inpt, self.W) + self.b  #[64, 10]
            
            tf.summary.histogram("weights", self.W)
            tf.summary.histogram("bias", self.b)

In [11]:
rnn = 'LSTM'

layer0 = input_layer(x, name='input_layer')

if rnn == 'LSTM':
    layer1 = LSTMcell(layer0.output, name='lstm_layer')
else:
    layer1 = vanilla_RNN(layer0.output, name='rnn_layer')

layer2 = output_layer(layer1.output, name='output_layer')
output = layer2.output

In [12]:
y_pred = tf.nn.softmax(output)
y_pred_cls = tf.argmax(y_pred, dimension=1)

In [13]:
#define name_scope
with tf.name_scope('cost'):
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=output, labels=y_true)
    cost = tf.reduce_mean(cross_entropy)
    
    #add the parameter cost to summary
    tf.summary.scalar('cost', cost)

In [14]:
optimizer = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)

In [15]:
#define name_scope
with tf.name_scope('accuracy'):
    correct_prediction = tf.equal(y_pred_cls, tf.argmax(y_true, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    #add accuracy to the summary
    tf.summary.scalar('accuracy', accuracy)

In [16]:
# specify a directory to save the model and the parameters
LOGDIR = '/tmp/mnist/lstm'

In [17]:
inpt = tf.reshape(x, shape=[-1, image_size, image_size, 1])
# Store 3 inputs in the summary
tf.summary.image('input', inpt, 3)

<tf.Tensor 'input:0' shape=() dtype=string>

In [18]:
import os
merged_summary = tf.summary.merge_all()
embedding_var = tf.Variable(tf.zeros([1024, 128]), name="embedding")

# specify which values should be used for embedding - the last fully connected layer output.
assignment = embedding_var.assign(layer1.output)

session = tf.Session()
session.run(tf.global_variables_initializer())
config = tf.contrib.tensorboard.plugins.projector.ProjectorConfig()
#create an object for the embedding class in config.
embedding = config.embeddings.add()
#Give embedding tensor the name that we created for the embedding_var
embedding.tensor_name = embedding_var.name
#set metadata_path to Path for .tsv file containing the true labels
embedding.metadata_path = os.path.join(os.getcwd(), 'data/labels_1024.tsv')
#set sprite image_path
embedding.sprite.image_path   = os.path.join(os.getcwd(),'data/sprite_1024.png')
# Now since all images are passed in a single file we need to specify the dimension of
# each image in that particular file so the the model can idetify them
embedding.sprite.single_image_dim.extend([28, 28])

# create a write instance to save in a particular file.
writer = tf.summary.FileWriter(LOGDIR)
# write the computation graph to the file
writer.add_graph(session.graph)
tf.contrib.tensorboard.plugins.projector.visualize_embeddings(writer, config)

In [19]:
batch_size = 200
epoch = 10
saver = tf.train.Saver()

In [20]:
import math
def train(epoch=10, batch_size=64):
    total_batch = math.ceil(float(len(data.train.images))/batch_size) 
    print '------------------TRAINING------------------'
    for i in range(epoch):
        avg_cost = 0

        session.run(assignment, feed_dict={x: data.test.images[:1024], y_true: data.test.labels[:1024]})
        saver.save(session, os.path.join(LOGDIR, "model.ckpt"), i)

        for j in range(int(total_batch)):
            x_batch, y_batch = data.train.next_batch(batch_size)
            
            feed_dict = {x: x_batch, y_true: y_batch}
            
            # at each batch calculate the summary and write to file.
            s = session.run(merged_summary, feed_dict)
            writer.add_summary(s, i*(total_batch)+j)                        

            _, c = session.run([optimizer, cost], feed_dict)                
            avg_cost += c / total_batch 
        print 'Epoch :{0} completed. Error :{1}'.format(i+1, avg_cost)
    print '-------------TRAINING COMPLETED-------------'    
    # save the final model
    saver.save(session, LOGDIR)

In [21]:
train(epoch=10, batch_size=batch_size)
session.close()

------------------TRAINING------------------
Epoch :1 completed. Error :0.328166353012
Epoch :2 completed. Error :0.10932832997
Epoch :3 completed. Error :0.102883451398
Epoch :4 completed. Error :0.0957934707573
Epoch :5 completed. Error :0.0876532530175
Epoch :6 completed. Error :0.0974973493273
Epoch :7 completed. Error :0.109495447109
Epoch :8 completed. Error :0.111430109272
Epoch :9 completed. Error :0.10511852601
Epoch :10 completed. Error :0.119582771611
-------------TRAINING COMPLETED-------------


In [22]:
print('Done training!')
print('Run `tensorboard --logdir=%s` in terminal and see the results in tensorboard.' % LOGDIR)

Done training!
Run `tensorboard --logdir=/tmp/mnist/lstm` in terminal and see the results in tensorboard.
